In [1]:
import warnings

import pandas as pd

from MLWrappers import SklearnBlackBox
from PrivacyAttacks import MiaPrivacyAttack, AloaPrivacyAttack
from MLPrivacyEvaluator import PrivacyEvaluator

warnings.simplefilter("ignore", UserWarning)

In [2]:
DS_NAME = 'adult'
DATA_FOLDER = f'./data/{DS_NAME}'

# we load the target black box model using our wrapper
target = SklearnBlackBox(f'./models/rf_{DS_NAME}.sav')

# We load the data used to train, test of the model, as well as the shadow data
train_set = pd.read_csv(f'{DATA_FOLDER}/{DS_NAME}_original_train_set.csv', skipinitialspace=True)[:800]
test_set = pd.read_csv(f'{DATA_FOLDER}/{DS_NAME}_original_test_set.csv', skipinitialspace=True)[:400]
shadow_set = pd.read_csv(f'{DATA_FOLDER}/{DS_NAME}_shadow_set.csv', skipinitialspace=True)[:500]

In [3]:
# We initialise the attacks, with the desired parameters for each
mia = MiaPrivacyAttack(target, n_shadow_models=5)
aloa = AloaPrivacyAttack(target, n_shadow_models=1, n_noise_samples_fit=200)
attacks = [mia, aloa]

In [4]:
# We initialise the PrivacyEvaluator object
# We pass the target model and the attacks we want to use
evaluator = PrivacyEvaluator(target, attacks)

In [5]:
# We use the fit() method to execute the attacks, starting from the shadow data
evaluator.fit(shadow_set)

Each shadow model uses 100.000 % of the data


100%|██████████| 500/500 [00:08<00:00, 57.46it/s]


In [6]:
# Then we can obtain the performances using the report() method
results = evaluator.report(train_set, test_set)
print(results)


ValueError: X has 2 features, but RandomForestClassifier is expecting 1 features as input.

In [ ]:
results['mia_attack']

{'classification_report': {'IN': {'precision': 0.715496368038741,
   'recall': 0.73875,
   'f1-score': 0.7269372693726938,
   'support': 800},
  'OUT': {'precision': 0.4411764705882353,
   'recall': 0.4125,
   'f1-score': 0.4263565891472868,
   'support': 400},
  'accuracy': 0.63,
  'macro avg': {'precision': 0.5783364193134881,
   'recall': 0.575625,
   'f1-score': 0.5766469292599903,
   'support': 1200},
  'weighted avg': {'precision': 0.6240564022219057,
   'recall': 0.63,
   'f1-score': 0.6267437092975581,
   'support': 1200}}}

In [ ]:
results['aloa_attack']['classification_report']

{'IN': {'precision': 0.6644295302013423,
  'recall': 0.99,
  'f1-score': 0.7951807228915664,
  'support': 800},
 'OUT': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 400},
 'accuracy': 0.66,
 'macro avg': {'precision': 0.33221476510067116,
  'recall': 0.495,
  'f1-score': 0.3975903614457832,
  'support': 1200},
 'weighted avg': {'precision': 0.44295302013422816,
  'recall': 0.66,
  'f1-score': 0.5301204819277109,
  'support': 1200}}